In [1]:
import sqlite3
from werkzeug.security import generate_password_hash
import pandas as pd
from transliterate import translit

In [2]:
def to_en(s):
    return translit(s.lower(), 'ru', reversed=True)

In [3]:
conn=sqlite3.connect('testFlask\db.sqlite')
c = conn.cursor()

In [4]:
def get_room(pid):
    c.execute("select * from place where id = {}".format(pid))
    _, _, res = c.fetchall()[0]
    c.execute("select * from room where id = {}".format(res))
    _, res, _, _, _, _, _ = c.fetchall()[0]
    return res

In [8]:
df = pd.read_csv('parser/users.csv', sep=';')[['id', 'email', 'password', 'surname', 'name', 'place_id']]\
.astype('str')
df['password'] = list(map(lambda x: generate_password_hash(x, method='sha256'), df['password']))
df['name'] = list(map(lambda x: '' if x == 'nan' else x, df['name']))

sur = df['surname']
name = df['name']
suren = list(map(to_en, sur))
namen = list(map(to_en, name))
email = df['email']
room = list(map(get_room, df['place_id']))
df['full_name'] = sur + ' ' + name + ' ' + sur + ' ' + suren + ' ' + namen + ' '\
    + suren + ' ' + room + ' ' + email;

df['full_name'] = list(map(lambda x: x.lower(), df['full_name']))

c.executemany("INSERT INTO user VALUES(?,?,?,?,?,?,?)", df.values)

In [5]:
df = pd.read_csv('parser/rooms.csv', sep=';')[['id', 'name', 'type', 'x', 'y', 'floor']].astype('str')

df['full_type'] = list(map(lambda x: x.lower(), df['type']))

c.executemany("INSERT INTO room VALUES(?,?,?,?,?,?,?)", df.values)

In [6]:
df = pd.read_csv('parser/places.csv', sep=';')[['id', 'number', 'room_id']].astype('str')
c.executemany("INSERT INTO place VALUES(?,?,?)", df.values)

In [9]:
c.close()
conn.commit()
conn.close()

In [11]:
c.execute('select * from user')
c.fetchall()

ProgrammingError: Cannot operate on a closed cursor.

In [12]:
df

,id,email,password,surname,name,place_id,full_name
0,1,petrov.ivan@company.com,sha256$PfgAsmhi$6536d104e80cfb258eaf63c8479032...,Петров,Иван,1,петров иван петров petrov ivan petrov 101 petr...
1,2,komandrov.sergej@company.com,sha256$gLCcCarx$c4c858f4d374e104ee177c7a7fb6c2...,Командров,Сергей,1,командров сергей командров komandrov sergej ko...
2,3,sapogov.artem@company.com,sha256$xFhbKXTq$a2decdc7124c25385a9d3c83ec2b97...,Сапогов,Артем,1,сапогов артем сапогов sapogov artem sapogov 10...
3,4,barista.@company.com,sha256$mirBeGjx$a879975d4fe09bbe7ec7a0b592eba0...,Бариста,,2,бариста бариста barista barista 102 barista....
4,5,oborudovanie.@company.com,sha256$NiNlRuHB$4b55c42e7402d5281ed078d95c6113...,Оборудование,,3,оборудование оборудование oborudovanie oboru...
...,...,...,...,...,...,...,...
156,157,ilin.innokentij@company.com,sha256$cB3GT4eE$4bcdcf249e48485c82430b82b82fb8...,Ильин,Иннокентий,119,ильин иннокентий ильин il'in innokentij il'in ...
157,158,schavov.mihail@company.com,sha256$mD0opqn5$8b135c7e995818eaca4dfdbf111f17...,Щавов,Михаил,119,щавов михаил щавов schavov mihail schavov 306а...
158,159,neverov.dmitrij@company.com,sha256$LPzIbtIk$8427eae1d5823e2cb830ef06251716...,Неверов,Дмитрий,119,неверов дмитрий неверов neverov dmitrij nevero...
159,160,kudrjavov.boris@company.com,sha256$JvUmpnbk$f0d74ec7fb1ab6d0b3c9ccca234d14...,Кудрявов,Борис,119,кудрявов борис кудрявов kudrjavov boris kudrja...
